# FORESEE - Massless dark ALP with photon-dark photon coupling

### Load Libraries 

In [ ]:
import numpy as np
import sys
import os

src_path = "../../src"
sys.path.append(src_path)
from foresee import Foresee, Utility, Model

from main import sigma_gpNucleus_aNucleus_analyt, sigma_aNucleus_gpNucleus_analyt, sigma_ae_gpe_log, sigma_gpe_ae_log
from constants import *

from timeit import default_timer as timer

from matplotlib import pyplot as plt
import matplotlib.tri as tri

plt.rc('text', usetex=True)
plt.rcParams['figure.dpi'] = 400

plt.rcParams['text.latex.preamble'] = [r"\usepackage{amsmath}"]
plt.rcParams['text.latex.preamble'] = [r"\usepackage{amssymb}"]
plt.rcParams['text.latex.preamble'] = [r"\usepackage{siunitx}"]
font = {'family': 'serif', 'serif': ['computer modern roman']}

plt.rc('font', **font)

SMALL_SIZE = 14
MEDIUM_SIZE = 18
BIGGER_SIZE = 20

plt.rc('font', size=MEDIUM_SIZE)  # controls default text sizes
plt.rc('axes', titlesize=BIGGER_SIZE)  # fontsize of the axes title
plt.rc('axes', labelsize=BIGGER_SIZE)  # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)  # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)  # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)  # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

## 1. Initialization 

In [ ]:
foresee = Foresee()

energy = "14"
modelname = "Dark_ALP_massless_a"
model = Model(modelname)

nsample = 250

num_of_masses = 22
num_of_masses_MATHUSLA = 22
masses = np.logspace(-3, 0.5, num_of_masses)
masses_MATHUSLA = np.logspace(-3, np.log10(0.030), num_of_masses_MATHUSLA)

num_of_couplings = 31
num_of_couplings_MATHUSLA = 21

### Generate LLP spectum

#### Mesons decays are the main production channel for dark axion-dark photon portal

The $a$, $\gamma^\prime$ can be produced in 3-body decays of pseudoscalar mesons $M \to \gamma \gamma^\prime$ via an offshell photon. 
<!-- For massive $a$ and $\gamma'$: -->
Pseudoscalar-meson decays:
\begin{equation}
\frac{d\text{BR}(M \!\to\! \gamma'a\gamma)}{dq^2 \, d\!\cos\theta}
 = \frac{g_{a\gamma\gamma^\prime}^2}{256 \pi ^2 m_M^6 q^6} \left(m_M^2-q^2\right)^3 (\cos(2\theta)+3) \left(-2 m_{\gamma^\prime}^2 \left(m_a^2+q^2\right)+m_{\gamma^\prime}^4+\left(m_a^2-q^2\right)^2\right)^{3/2} \text{BR}(M \!\to \!\gamma\gamma).
\end{equation}

In this notebook, we put $m_a=0$. The other one considers  $m_{\gamma'}=0$.

PDG codes: $\gamma$ = 22, $\pi_0$ = 111, $\eta$ = 221, $\eta^\prime$ = 331.

<!-- To generate LLP spectrum, use Dark_ALP_massless_a.py script.   -->


Vector-meson decays:

\begin{equation} 
	\begin{aligned}
		\!\!\text{Dark ALP:}\
			&\frac{{\rm BR}_{V \rightarrow a \gamma^\prime}}{{\rm BR}_{V \rightarrow ee}} \!=\! \frac{ g_{a\gamma\gamma^\prime}^2 (M-m_a-m_{\gamma^\prime}) (M+m_a-m_{\gamma^\prime}) (M-m_a+m_{\gamma^\prime}) (M+m_a+m_{\gamma^\prime}) \sqrt{\left(-M^2+m_a^2+m_{\gamma^\prime}^2\right)^2-4 m_a^2 m_{\gamma^\prime}^2}}{32 \pi  \alpha_{\text{EM}} M \sqrt{M^2-4 m_e^2} \left(M^2+2 m_e^2\right)}, \!\! \\
	\end{aligned}
\end{equation} 

In [ ]:
# model.add_production_3bodydecay(
#     pid0="111", # pi0, m_pi0=0.135, br=0.98823
#     pid1="22",  # photon
#     pid2="22",  # a - we put m_a=0; mass(pid3) = mass = m_gprime
#     br="0.98823 * coupling**2 / 256. / 3.1415**2 / q**6 / 0.135**6 * (0.135**2-q**2)**3 * (np.cos(2*th)+3) * (-2*mass**2*q**2 + mass**4 + q**4)**1.5",generator="EPOSLHC",
#     energy=energy,
#     nsample=nsample,
# )

# model.add_production_3bodydecay(
#     pid0="221", # eta, m_eta=0.547, br=0.3931
#     pid1="22",
#     pid2="22",
#     br="0.3931 * coupling**2 / 256. / 3.1415**2 / q**6 / 0.547**6 * (0.547**2-q**2)**3 * (np.cos(2*th)+3) * (-2*mass**2*q**2 + mass**4 + q**4)**1.5",
#     generator="EPOSLHC",
#     energy=energy,
#     nsample=nsample,
# )

# model.add_production_3bodydecay(
#     pid0="331", # etaprime, m_etaprime=0.957, br=0.222
#     pid1="22",
#     pid2="22",
#     br="0.222 * coupling**2 / 256. / 3.1415**2 / q**6 / 0.957**6 * (0.957**2-q**2)**3 * (np.cos(2*th)+3) * (-2*mass**2*q**2 + mass**4 + q**4)**1.5",
#     generator="EPOSLHC",
#     energy=energy,
#     nsample=nsample,
# )

In [ ]:
# V(p0) -> a(p1) + gprime(p2)
# p1**2 = m1**2 = m_a**2
# p2**2 = m2**2 = m_gprime**2

model.add_production_2bodydecay(
    pid0 = "113", # rho
    pid1 = "0",   # pid1 = 0 means mass_pid1 = m1 = mass_llp0;   mass_pid2 = m2 = mass = mass_llp1
    br = "4.72e-5 * coupling**2 * (1 * (m0 - m1 - m2)*(m0 + m1 - m2)*(m0 - m1 + m2)* (m0 + m1 + m2)*np.sqrt(-4*m1**2*m2**2 + (-m0**2 + m1**2 + m2**2)**2)) / (32.* ALPHAEM * m0 * np.sqrt(m0**2 - 4*M_ELECTRON**2) * (m0**2 + 2*M_ELECTRON**2) * np.pi)",
    generator = "EPOSLHC",
    energy = energy,
    nsample = nsample,
)

model.add_production_2bodydecay(
   pid0 = "223", # omega
   pid1 = "0",
   br = "7.38e-5 * coupling**2 * (1 * (m0 - m1 - m2)*(m0 + m1 - m2)*(m0 - m1 + m2)* (m0 + m1 + m2)*np.sqrt(-4*m1**2*m2**2 + (-m0**2 + m1**2 + m2**2)**2)) / (32.* ALPHAEM * m0 * np.sqrt(m0**2 - 4*M_ELECTRON**2) * (m0**2 + 2*M_ELECTRON**2) * np.pi)",
   generator = "EPOSLHC",
   energy = energy,
   nsample = nsample,
)

model.add_production_2bodydecay(
   pid0 = "333", # phi
   pid1 = "0",
   br = "2.98e-4 * coupling**2 * (1 * (m0 - m1 - m2)*(m0 + m1 - m2)*(m0 - m1 + m2)* (m0 + m1 + m2)*np.sqrt(-4*m1**2*m2**2 + (-m0**2 + m1**2 + m2**2)**2)) / (32.* ALPHAEM * m0 * np.sqrt(m0**2 - 4*M_ELECTRON**2) * (m0**2 + 2*M_ELECTRON**2) * np.pi)",
   generator = "EPOSLHC",
   energy = energy,
   nsample = nsample,
)

model.add_production_2bodydecay(
    pid0 = "443", # J/ψ
    pid1 = "0",
    br = "0.0597 * coupling**2 * (1 * (m0 - m1 - m2)*(m0 + m1 - m2)*(m0 - m1 + m2)* (m0 + m1 + m2)*np.sqrt(-4*m1**2*m2**2 + (-m0**2 + m1**2 + m2**2)**2)) / (32.* ALPHAEM * m0 * np.sqrt(m0**2 - 4*M_ELECTRON**2) * (m0**2 + 2*M_ELECTRON**2) * np.pi)",
    generator = "Pythia8",
    energy = energy,
    nsample = nsample,
)

# model.add_production_2bodydecay(
#    pid0 = "100443", # \psi(2S)
#    pid1 = "0",
#    br = "0.00993 * coupling**2 * (1 * (m0 - m1 - m2)*(m0 + m1 - m2)*(m0 - m1 + m2)* (m0 + m1 + m2)*np.sqrt(-4*m1**2*m2**2 + (-m0**2 + m1**2 + m2**2)**2)) / (32.* ALPHAEM * m0 * np.sqrt(m0**2 - 4*M_ELECTRON**2) * (m0**2 + 2*M_ELECTRON**2) * np.pi)",   
#    generator = "Pythia8",
#    energy = energy,
#    nsample = nsample,
# )

# model.add_production_2bodydecay(
#     pid0 = "553", # Υ ($\Upsilon(1S)$)
#     pid1 = "0",
#     br = "0.0238 * coupling**2 * (1 * (m0 - m1 - m2)*(m0 + m1 - m2)*(m0 - m1 + m2)* (m0 + m1 + m2)*np.sqrt(-4*m1**2*m2**2 + (-m0**2 + m1**2 + m2**2)**2)) / (32.* ALPHAEM * m0 * np.sqrt(m0**2 - 4*M_ELECTRON**2) * (m0**2 + 2*M_ELECTRON**2) * np.pi)",    
#     generator = "Pythia8",
#     energy = energy,
#     nsample = nsample,
# )

# model.add_production_2bodydecay(
#    pid0 = "100553", # $\Upsilon(2S)$
#    pid1 = "0",
#    br = "0.0191 * coupling**2 * (1 * (m0 - m1 - m2)*(m0 + m1 - m2)*(m0 - m1 + m2)* (m0 + m1 + m2)*np.sqrt(-4*m1**2*m2**2 + (-m0**2 + m1**2 + m2**2)**2)) / (32.* ALPHAEM * m0 * np.sqrt(m0**2 - 4*M_ELECTRON**2) * (m0**2 + 2*M_ELECTRON**2) * np.pi)",
#    generator = "Pythia8",
#    energy = energy,
#    nsample = nsample,
# )

# model.add_production_2bodydecay(
#    pid0 = "200553", # $\Upsilon(3S)$
#    pid1 = "0",
#    br = "0.0218 * coupling**2 * (1 * (m0 - m1 - m2)*(m0 + m1 - m2)*(m0 - m1 + m2)* (m0 + m1 + m2)*np.sqrt(-4*m1**2*m2**2 + (-m0**2 + m1**2 + m2**2)**2)) / (32.* ALPHAEM * m0 * np.sqrt(m0**2 - 4*M_ELECTRON**2) * (m0**2 + 2*M_ELECTRON**2) * np.pi)",
#    generator = "Pythia8",
#    energy = energy,
#    nsample = nsample,
# )

In [ ]:
model.set_ctau_1d(filename="model/ctau_gprime.txt", coupling_ref=1)

branchings = [
    [ "BR_gammaprime_agamma", "black", "solid", r"$a\gamma$", 0.110, 0.30 ],
    [ "BR_gammaprime_aee", "red", "solid", r"$a e^+ e^-$", 0.110, 0.012],
]

model.set_br_1d(
    modes=[channel for channel, _, _, _, _, _ in branchings],
    filenames=[ "model/br/" + channel + ".txt" for channel, _, _, _, _, _ in branchings ],
)

foresee.set_model(model=model)

In [ ]:
# %matplotlib inline

# mass_llp0 = 0.0 # m_a
# mass_llp1 = 0.1 # m_gprime

# plt_1, plt_2 = foresee.get_llp_spectrum(mass=mass_llp1, mass_llp0=mass_llp0, coupling=1e-4, do_plot=True, save_file=False)
# plt_1.savefig("./output/test_LLP_spect_plt_1.pdf")
# plt_2.savefig("./output/test_LLP_spect_plt_2.pdf")

# plt_1.show()
# plt_2.show()


# plt_1, plt_2 = foresee.get_llp_spectrum(mass=mass_llp1, mass_llp0=mass_llp0, coupling=1e-4, do_plot=True, save_file=False, detector="MATHUSLA")
# plt_1.savefig("./output/test_LLP_spect_plt_1_MATHUSLA.pdf")
# plt_2.savefig("./output/test_LLP_spect_plt_2_MATHUSLA.pdf")

# plt_1.show()
# plt_2.show()

In [ ]:
# from timeit import default_timer as timer

# for count, mass in enumerate(masses):
#     mass_llp0 = 0.0    # m_a
#     mass_llp1 = mass   # m_gprime

#     start = timer()

#     foresee.get_llp_spectrum(mass=mass_llp1, mass_llp0=mass_llp0, coupling=1)

#     end = timer()
#     time_length_sec = end - start

#     count += 1
#     time_length_sec_total = time_length_sec * num_of_masses
#     print("%.2f%% done, " % float(count / num_of_masses * 100), "approx. total run time : %.1f m, " % float(time_length_sec_total / 60), "approx. waiting time: %.1f m" % float(time_length_sec_total * (1 - count / num_of_masses) / 60))


# count_total = num_of_masses_MATHUSLA
# for count, mass in enumerate(masses_MATHUSLA):
#     mass_llp0 = 0.0
#     mass_llp1 = mass

#     start = timer()

#     foresee.get_llp_spectrum(mass=mass_llp1, mass_llp0=mass_llp0, coupling=1, detector="MATHUSLA")

#     end = timer()
#     time_length_sec = end - start

#     count += 1
#     time_length_sec_total = time_length_sec * count_total
#     print("%.2f%% done, " % float(count/count_total * 100), "approx. total run time : %.1f m, " % float(time_length_sec_total/60), "approx. waiting time: %.1f m" % float(time_length_sec_total * (1 - count / count_total) / 60))

In [ ]:
# productions = [
#     ["111"    , None      , "firebrick"   , r"$\pi$"         ],   
#     # ["221"    , None      , "red"         , r"$\eta$"        ],   
#     # ["331"    , None , "salmon"      , r"$\eta'$"       ],  
#     ["113"    , None , "dodgerblue"  , r"$\rho$"        ],   
#     ["223"    , None , "blue"        , r"$\omega$"      ],   
#     ["333"    , None , "deepskyblue" , r"$\phi$"        ],  
#     ["443"    , None  , "gold"        , r"$J/\psi$"      ],   
#     ["100443" , None  , "orange"      , r"$\psi(2S)$"    ],  
#     ["553"    , None  , "green"       , r"$\Upsilon(1S)$"],   
#     ["100553" , None  , "limegreen"   , r"$\Upsilon(2S)$"],  
#     ["200553" , None  , "lime"        , r"$\Upsilon(3S)$"],  
# ]

# plot = foresee.plot_production(
#     masses = masses, 
#     productions = productions,
#     condition="True", 
#     xlims=[0.03,1],
#     ylims=[10**0,10**7], 
#     xlabel=r"$m_{a}$ [GeV]", 
#     ylabel=r"Production rate $\sigma \times g_\gamma^2$ [pb]",
#     legendloc=(1.02,1.02),
#     fs_label=12,
#     dolegend=True,
# )

# plot.subplots_adjust(left=0.11, right=0.95, bottom=0.12, top=0.97)
# plot.savefig("output/Dark_ALP_massless_a_Production_channels_FASER.pdf")
# plot.show()



# productions = [
#     ["111"    , None      , "firebrick"   , r"$\pi$"         ],   
#     # ["221"    , None      , "red"         , r"$\eta$"        ],   
#     # ["331"    , None , "salmon"      , r"$\eta'$"       ],  
#     ["113"    , None , "dodgerblue"  , r"$\rho$"        ],   
#     ["223"    , None , "blue"        , r"$\omega$"      ],   
#     ["333"    , None , "deepskyblue" , r"$\phi$"        ],  
#     ["443"    , None  , "gold"        , r"$J/\psi$"      ],   
#     ["100443" , None  , "orange"      , r"$\psi(2S)$"    ],  
#     ["553"    , None  , "green"       , r"$\Upsilon(1S)$"],   
#     ["100553" , None  , "limegreen"   , r"$\Upsilon(2S)$"],  
#     ["200553" , None  , "lime"        , r"$\Upsilon(3S)$"],  
# ]

# plot = foresee.plot_production(
#     masses = masses_MATHUSLA, 
#     productions = productions,
#     condition="True", 
#     xlims=[0.03,0.1],
#     ylims=[1e2,1e7], 
#     xlabel=r"$m_{a}$ [GeV]", 
#     ylabel=r"Production rate $\sigma \times g_\gamma^2$ [pb]",
#     legendloc=(1.02,1.02),
#     fs_label=12,
#     dolegend=True,
#     energy=energy,
#     detector="MATHUSLA",
# )

# plot.subplots_adjust(left=0.11, right=0.95, bottom=0.12, top=0.97)
# plot.savefig("output/Dark_ALP_massless_a_Production_channels_MATHUSLA.pdf")
# plot.show()

## 2. No FPF + MATHUSLA

### Primary production

- FASER2 - the nominal/default setup -  $\gamma^\prime\to a \gamma$ with $BR=1$

In [ ]:
luminosity, distance = 3000, 480
setup, selection, channels, length = "FASER2_agamma", "np.sqrt(x.x**2 + x.y**2)< 1*480./480.", [ "BR_gammaprime_agamma" ], 5
foresee.set_detector(length=length,
                     selection=selection,
                     channels=channels,
                     distance=distance,
                     luminosity=luminosity)


list_nevents = []
for mass in masses:
    couplings, _, nevents, _, _ = foresee.get_events(mass=mass, energy=energy, couplings=np.logspace(-6, -2, num_of_couplings))
    list_nevents.append(nevents)
np.save("model/results/" + energy + "TeV_" + setup + ".npy", [masses, couplings, list_nevents])

- FASER2 setup with decay $\gamma^\prime\to a ee$

In [ ]:
luminosity, distance = 3000, 480
setup, selection, channels, length = "FASER2_aee", "np.sqrt(x.x**2 + x.y**2)< 1*480./480.", [ "BR_gammaprime_aee" ], 5
foresee.set_detector(length=length,
                     selection=selection,
                     channels=channels,
                     distance=distance,
                     luminosity=luminosity)


list_nevents = []
for mass in masses:
    couplings, _, nevents, _, _ = foresee.get_events(mass=mass, energy=energy, couplings=np.logspace(-6, -2, num_of_couplings))
    list_nevents.append(nevents)
np.save("model/results/" + energy + "TeV_" + setup + ".npy", [masses, couplings, list_nevents])

In [ ]:
luminosity, distance = 3000, 480 - 2  # L=2m
setup, selection, channels, length = "FASERnu2_agamma", "np.sqrt(x.x**2 + x.y**2)< 0.25", [ "BR_gammaprime_agamma" ], 2
foresee.set_detector(length=length,
                     selection=selection,
                     channels=channels,
                     distance=distance,
                     luminosity=luminosity)

#get reach
list_nevents = []
for mass in masses:
    couplings, _, nevents, _, _ = foresee.get_events(
        mass=mass,
        energy=energy,
        couplings=np.logspace(-6, -2, num_of_couplings),
        preselectioncuts="th<0.01 and p>1000")
    list_nevents.append(nevents)

#save results
np.save("model/results/" + energy + "TeV_" + setup + ".npy", [masses, couplings, list_nevents])

### Secondary Production

- FASER2 - the nominal/default setup -  $\gamma^\prime\to a \gamma$ with $BR=1$

In [ ]:
luminosity, distance = 3000, 480
setup, selection, channels, length = "FASER2_agamma_secondary", "np.sqrt(x.x**2 + x.y**2)< 1*480./480.", [ "BR_gammaprime_agamma" ], 5
foresee.set_detector(length=length,
                     selection=selection,
                     channels=channels,
                     distance=distance,
                     luminosity=luminosity)


list_nevents = []
for mass in masses:
    couplings, _, nevents, _, _ = foresee.get_events_secondary_NOFPF(
        mass=mass,
        energy=energy,
        couplings=np.logspace(-6, -2, num_of_couplings),
        preselectioncuts="th<0.01 and p>100",
        which_LLP="llp0",
        sign="decay_F2",  
        coup_ref=1,        
        m1=0.0,
        m3=mass,
        Z=Z_TUNGSTEN,
        A=A_TUNGSTEN,
        RHO=RHO_TUNGSTEN,
        sigma_secondary_LLP2_LLP3=sigma_aNucleus_gpNucleus_analyt,
    )
    list_nevents.append(nevents)
    # print(nevents)
np.save("model/results/" + energy + "TeV_" + setup + ".npy", [masses, couplings, list_nevents])

- FASER $\nu2$ setup with decay  $\gamma^\prime\to a \gamma$

In [ ]:

luminosity, distance = 3000, 480
setup, selection, channels, length = "FASERnu2_agamma_secondary", "np.sqrt(x.x**2 + x.y**2)< 0.25", [ "BR_gammaprime_agamma" ], 2
foresee.set_detector(length=length,
                     selection=selection,
                     channels=channels,
                     distance=distance,
                     luminosity=luminosity)


list_nevents = []
for mass in masses:
    couplings, _, nevents, _, _ = foresee.get_events_secondary_NOFPF(
        mass=mass,
        energy=energy,
        couplings=np.logspace(-6, -2, num_of_couplings),
        preselectioncuts="th<0.01 and p>1000",
        which_LLP="llp0",
        sign="decay_Fnu2",  
        coup_ref=1,
        m1=0.0,
        m3=mass,
        Z=Z_TUNGSTEN,
        A=A_TUNGSTEN,
        RHO=RHO_TUNGSTEN,
        sigma_secondary_LLP2_LLP3=sigma_aNucleus_gpNucleus_analyt,
    )
    list_nevents.append(nevents)
np.save("model/results/" + energy + "TeV_" + setup + ".npy", [masses, couplings, list_nevents])

- FASER $\nu2$ setup with scattering and decay outside F2

In [ ]:
luminosity, distance = 3000, 480
setup, selection, channels, length = "FASERnu2_scat", "np.sqrt(x.x**2 + x.y**2)< 0.25", [ "BR_gammaprime_agamma" ], 2
foresee.set_detector(length=length,
                     selection=selection,
                     channels=channels,
                     distance=distance,
                     luminosity=luminosity)


list_nevents = []
for mass in masses:
    couplings, _, nevents, _, _ = foresee.get_events_secondary_NOFPF(
        mass=mass,
        energy=energy,
        couplings=np.logspace(-6, -2, num_of_couplings),
        preselectioncuts="th<0.01",
        which_LLP="llp0",
        sign="scat_e",  
        coup_ref=1,
        m1=0.0,
        m3=mass,
        Z=Z_TUNGSTEN,
        A=A_TUNGSTEN,
        RHO=RHO_TUNGSTEN,
        sigma_secondary_LLP2_LLP3=sigma_ae_gpe_log,
        sigma_secondary_LLP3_LLP2=sigma_gpe_ae_log,
        # sigma_secondary_LLP2_LLP3=sigma_ae_gpe_analyt,
        # sigma_secondary_LLP3_LLP2=sigma_gpe_ae_analyt,
    )
    list_nevents.append(nevents)
np.save("model/results/" + energy + "TeV_" + setup + ".npy", [masses, couplings, list_nevents])

### MATHUSLA

- MATHUSLA setup with decay  $\gamma^\prime\to a ee$

In [ ]:
num_of_couplings_MATHUSLA = 21

In [ ]:
luminosity = 3000
setup, channels = "MATHUSLA_aee", ["BR_gammaprime_aee"]

foresee.set_detector( length=0.0, channels=channels, distance=0.0, luminosity=luminosity)  # for MATHUSLA, we need to modify the foresee code


list_nevents = []
for mass in masses_MATHUSLA[::2]:
    couplings, _, nevents, _, _ = foresee.get_events(mass=mass, energy=energy, couplings=np.logspace(-4, -2, num_of_couplings_MATHUSLA), preselectioncuts="p>2.0", detector="MATHUSLA")
    list_nevents.append(nevents)
np.save("model/results/" + energy + "TeV_" + setup + ".npy", [masses_MATHUSLA[::2], couplings, list_nevents])

In [ ]:
# luminosity = 3000
# setup, channels = "MATHUSLA_aee_secondary", ["BR_gammaprime_aee"]
# foresee.set_detector(length=0.0,
#                      channels=channels,
#                      distance=0.0,
#                      luminosity=luminosity)


# list_nevents = []
# for mass in masses_MATHUSLA:
#     couplings, _, nevents, _, _ = foresee.get_events_secondary_MATHUSLA(
#         mass=mass,
#         energy=energy,
#         couplings=np.logspace(-6, -2, num_of_couplings_MATHUSLA),
#         preselectioncuts="p>2.0",
#         m1=0.0,
#         m3=mass,
#         sigma_secondary_LLP2_LLP3=sigma_aNucleus_gpNucleus_analyt,
#     )
#     list_nevents.append(nevents)
# np.save("model/results/" + energy + "TeV_" + setup + ".npy", [masses_MATHUSLA, couplings, list_nevents])

## 2'. FPF

#### Primary

- FASER2 - the nominal/default setup -  $\gamma^\prime\to a \gamma$ with $BR=1$

In [ ]:
luminosity, distance = 3000, 620
setup, selection, channels, length = "FPF_FASER2_agamma", "np.sqrt(x.x**2 + x.y**2)< 1.", [ "BR_gammaprime_agamma" ], 25
foresee.set_detector(length=length,
                     selection=selection,
                     channels=channels,
                     distance=distance,
                     luminosity=luminosity)


list_nevents = []
for mass in masses:
    couplings, _, nevents, _, _ = foresee.get_events(mass=mass, energy=energy, couplings=np.logspace(-6, -2, num_of_couplings))
    list_nevents.append(nevents)
np.save("model/results/" + energy + "TeV_" + setup + ".npy", [masses, couplings, list_nevents])

- FASER2 setup with decay $\gamma^\prime\to a ee$

In [ ]:
luminosity, distance = 3000, 620
setup, selection, channels, length = "FPF_FASER2_aee", "np.sqrt(x.x**2 + x.y**2)< 1.", [ "BR_gammaprime_aee" ], 25
foresee.set_detector(length=length,
                     selection=selection,
                     channels=channels,
                     distance=distance,
                     luminosity=luminosity)


list_nevents = []
for mass in masses:
    couplings, _, nevents, _, _ = foresee.get_events(mass=mass, energy=energy, couplings=np.logspace(-6, -2, num_of_couplings))
    list_nevents.append(nevents)
np.save("model/results/" + energy + "TeV_" + setup + ".npy", [masses, couplings, list_nevents])

- FASER $\nu2$ setup with decay  $\gamma^\prime\to a \gamma$

In [ ]:
luminosity, distance = 3000, 620 - 8  # L=8m
setup, selection, channels, length = "FPF_FASERnu2_agamma", "np.sqrt(x.x**2 + x.y**2)< 0.2", [ "BR_gammaprime_agamma" ], 8
foresee.set_detector(length=length,
                     selection=selection,
                     channels=channels,
                     distance=distance,
                     luminosity=luminosity)


list_nevents = []
for mass in masses:
    couplings, _, nevents, _, _ = foresee.get_events( mass=mass, energy=energy, couplings=np.logspace(-6, -2, num_of_couplings), preselectioncuts="th<0.01 and p>1000")
    list_nevents.append(nevents)
#save results
np.save("model/results/" + energy + "TeV_" + setup + ".npy", [masses, couplings, list_nevents])

#### Secondary

- FASER2 - the nominal/default setup -  $\gamma^\prime\to a \gamma$ with $BR=1$

In [ ]:
luminosity, distance = 3000, 620
setup, selection, channels, length = "FPF_FASER2_agamma_secondary", "np.sqrt(x.x**2 + x.y**2)< 1.", [ "BR_gammaprime_agamma" ], 25
foresee.set_detector(length=length,
                     selection=selection,
                     channels=channels,
                     distance=distance,
                     luminosity=luminosity)


list_nevents = []
for mass in masses:
    couplings, _, nevents, _, _ = foresee.get_events_secondary_FPF(
        mass=mass,
        energy=energy,
        couplings=np.logspace(-6, -2, num_of_couplings),
        # couplings=np.logspace(-3, 10, num_of_couplings),
        preselectioncuts="th<0.01 and p>100",
        which_LLP="llp0",
        sign="decay_F2",  
        coup_ref=1,        
        m1=0.0,
        m3=mass,
        Z=Z_TUNGSTEN,
        A=A_TUNGSTEN,
        RHO=RHO_TUNGSTEN,
        sigma_secondary_LLP2_LLP3=sigma_aNucleus_gpNucleus_analyt,
        FLARE_upscat=False
    )
    list_nevents.append(nevents)
    # print(nevents)
np.save("model/results/" + energy + "TeV_" + setup + ".npy", [masses, couplings, list_nevents])

- FASER2 - the nominal/default setup -  $\gamma^\prime\to a \gamma$ with $BR=1$ - with FLARE upscattering

In [ ]:
luminosity, distance = 3000, 620
setup, selection, channels, length = "FPF_FASER2_agamma_secondary_with_FLARE", "np.sqrt(x.x**2 + x.y**2)< 1.", [ "BR_gammaprime_agamma" ], 25
foresee.set_detector(length=length,
                     selection=selection,
                     channels=channels,
                     distance=distance,
                     luminosity=luminosity)


list_nevents = []
for mass in masses:
    couplings, _, nevents, _, _ = foresee.get_events_secondary_FPF(
        mass=mass,
        energy=energy,
        couplings=np.logspace(-6, -2, num_of_couplings),
        preselectioncuts="th<0.01 and p>100",
        which_LLP="llp0",
        sign="decay_F2",  
        coup_ref=1,        
        m1=0.0,
        m3=mass,
        Z=Z_TUNGSTEN,
        A=A_TUNGSTEN,
        RHO=RHO_TUNGSTEN,
        sigma_secondary_LLP2_LLP3=sigma_aNucleus_gpNucleus_analyt,
        sigma_secondary_LLP3_LLP2_or_LLP2_LLP3_FLARE=sigma_aNucleus_gpNucleus_analyt,
        FLARE_upscat=True
    )
    list_nevents.append(nevents)
np.save("model/results/" + energy + "TeV_" + setup + ".npy", [masses, couplings, list_nevents])

- FASER2 setup with decay $\gamma^\prime\to a ee$

In [ ]:
# #
# luminosity, distance = 3000, 620
# setup, selection, channels, length = "FPF_FASER2_aee_secondary", "np.sqrt(x.x**2 + x.y**2)< 1.", [ "BR_gammaprime_aee" ], 25
# foresee.set_detector(length=length,
#                      selection=selection,
#                      channels=channels,
#                      distance=distance,
#                      luminosity=luminosity)


# list_nevents = []
# for mass in masses:
#     couplings, _, nevents, _, _ = foresee.get_events_secondary_FPF(
#         mass=mass,
#         energy=energy,
#         couplings=np.logspace(-6, -2, num_of_couplings),
#         preselectioncuts="th<0.01 and p>100",
#         which_LLP="llp0",
#         sign="decay_F2",  
#         coup_ref=1,        
#         m1=0.0,
#         m3=mass,
#         Z=Z_TUNGSTEN,
#         A=A_TUNGSTEN,
#         RHO=RHO_TUNGSTEN,
#         sigma_secondary_LLP2_LLP3=sigma_aNucleus_gpNucleus_analyt,
#     )
#     list_nevents.append(nevents)
# np.save("model/results/" + energy + "TeV_" + setup + ".npy", [masses, couplings, list_nevents])

- FASER $\nu2$ setup with decay  $\gamma^\prime\to a \gamma$

In [ ]:
#
luminosity, distance = 3000, 610
setup, selection, channels, length = "FPF_FASERnu2_agamma_secondary", "np.sqrt(x.x**2 + x.y**2)< 0.25", [ "BR_gammaprime_agamma" ], 8
foresee.set_detector(length=length,
                     selection=selection,
                     channels=channels,
                     distance=distance,
                     luminosity=luminosity)


list_nevents = []
for mass in masses:
    couplings, _, nevents, _, _ = foresee.get_events_secondary_FPF(
        mass=mass,
        energy=energy,
        couplings=np.logspace(-6, -2, num_of_couplings),
        preselectioncuts="th<0.01 and p>1000",
        which_LLP="llp0",
        sign="decay_Fnu2",  
        coup_ref=1,
        m1=0.0,
        m3=mass,
        Z=Z_TUNGSTEN,
        A=A_TUNGSTEN,
        RHO=RHO_TUNGSTEN,
        sigma_secondary_LLP2_LLP3=sigma_aNucleus_gpNucleus_analyt,
    )
    list_nevents.append(nevents)
np.save("model/results/" + energy + "TeV_" + setup + ".npy", [masses, couplings, list_nevents])

- FASER $\nu2$ setup with scattering and decay outside F2

In [ ]:
luminosity, distance = 3000, 610
setup, selection, channels, length = "FPF_FASERnu2_scat", "np.sqrt(x.x**2 + x.y**2)< 0.25", [ "BR_gammaprime_agamma" ], 8
foresee.set_detector(length=length,
                     selection=selection,
                     channels=channels,
                     distance=distance,
                     luminosity=luminosity)


list_nevents = []
for mass in masses:
    couplings, _, nevents, _, _ = foresee.get_events_secondary_FPF(
        mass=mass,
        energy=energy,
        couplings=np.logspace(-6, -2, num_of_couplings),
        preselectioncuts="th<0.01",
        which_LLP="llp0",
        sign="scat_e",  
        coup_ref=1,
        m1=0.0,
        m3=mass,
        Z=Z_TUNGSTEN,
        A=A_TUNGSTEN,
        RHO=RHO_TUNGSTEN,
        sigma_secondary_LLP2_LLP3=sigma_ae_gpe_log,
        sigma_secondary_LLP3_LLP2_or_LLP2_LLP3_FLARE=sigma_gpe_ae_log,
    )
    list_nevents.append(nevents)
np.save("model/results/" + energy + "TeV_" + setup + ".npy", [masses, couplings, list_nevents])

- FLARE - e scattering

In [ ]:

luminosity, distance = 3000, 600
setup, selection, channels, length = "FPF_FLARE_scat", "np.sqrt(x.x**2 + x.y**2)< 0.5", [ "BR_gammaprime_agamma" ], 7
foresee.set_detector(length=length,
                     selection=selection,
                     channels=channels,
                     distance=distance,
                     luminosity=luminosity)


list_nevents = []
for mass in masses:
    couplings, _, nevents, _, _ = foresee.get_events_secondary_FPF(
        mass=mass,
        energy=energy,
        couplings=np.logspace(-6, -2, num_of_couplings),
        preselectioncuts="th<0.01",
        which_LLP="llp0",
        sign="scat_FLARE",  
        coup_ref=1,
        m1=0.0,
        m3=mass,
        Z=Z_LAr,
        A=A_LAr,
        RHO=RHO_LAr,
        sigma_secondary_LLP2_LLP3=sigma_ae_gpe_log,
        sigma_secondary_LLP3_LLP2_or_LLP2_LLP3_FLARE=sigma_gpe_ae_log,
    )
    list_nevents.append(nevents)
np.save("model/results/" + energy + "TeV_" + setup + ".npy", [masses, couplings, list_nevents])

## 3. Plot the Results

In [ ]:
# NuCal NoE 90% CL
5 - 3.5 + 1.28 * np.sqrt(5)

In [ ]:
setups = [
    [ "14TeV_FASER2_agamma.npy", r"FASER2 ($E_{a\gamma}> 0.1$ TeV)", "black", "solid", 0., 3 ],
    [ "14TeV_FASER2_aee.npy", "FASER2 ($E_{ae^+e^-}> 0.1$ TeV)", "red", "solid", 0., 3 ], 
    [ "14TeV_FASERnu2_agamma.npy", r"FASER$\nu2$ ($E_{a\gamma}> 1$ TeV)", "black", "dotted", 0., 3 ],

    [ "14TeV_FASER2_agamma_secondary.npy", "FASER2 (sec., $E_{a\gamma}> 0.1$ TeV)", "black", "dashed", 0., 3 ],
    [ "14TeV_FASERnu2_agamma_secondary.npy", r"FASER$\nu2$ (sec., $E_{a\gamma}> 1$ TeV)", "black", "dashdot", 0., 3 ],

    [ "14TeV_FASERnu2_scat.npy", r"FASER$\nu2$ ($e^-$ scat., decay outside)", "gold", "solid", 0., 20 ],

    [ "14TeV_MATHUSLA_aee.npy", r"MATHUSLA ($E_{a e^+ e^-}> 2$ GeV)", "brown", "solid", 0., 3],

    [ "0.4TeV_SHiP_agamma.npy", r"SHiP ($E_{a \gamma}> 2$ GeV)", "darkgreen", "dashed", 0., 100 * 2],    
]

In [ ]:
bounds = [
    ["Heavy_dark_photon_BaBar.txt", "BaBar", 0.55, 2.45 * 10**-3, 0],
    ["0.069TeV_NuCal_agamma.npy.txt", "NuCal", 0.012, 4.8e-1 * 10**-4, -20],
    ["0.4TeV_CHARM_agamma.npy.txt", "CHARM", 0.012, 7.6e-1 * 10**-4, -20],
]

In [ ]:
projections = [
    [ "Heavy_dark_photon_Belle_2_50fb_inverse.txt", "blue", "Belle2", 0.55, 9e-5, 0 ],
]

In [ ]:
plt, ax, ax2 = foresee.plot_reach(
    setups=setups,
    bounds=bounds,
    projections=projections,
    xlims=[0.001, 1],
    ylims=[5e-7, 10**-2],
    xlabel=r"$m_{\gamma^\prime}$ [GeV]",
    ylabel=r"$g_{a\gamma\gamma^\prime}$ [1/GeV]",
    legendloc=(1.00, 0.28),
    branchings=branchings,
    figsize=(8, 8),
    save_file=True,
)

ax.text(0.020, 2.0 * 10**-6, "SN1987", alpha=0.3, color='blue', fontsize=12)
low = np.loadtxt("model/lines/Heavy_dark_photon_Supernova_Emission_low.txt")
high = np.loadtxt("model/lines/Heavy_dark_photon_Supernova_Emission_high.txt")
x = np.geomspace(0.001, 1, num_of_couplings)
y = np.interp(x, high[:, 0], high[:, 1])
z = np.interp(x, low[:, 0], low[:, 1])
ax.fill_between(x, y, z, where=y >= z, alpha=0.05, color='blue', interpolate=True, lw=1, zorder=-10)

ax.text(0.00105, 3.0 * 10**-6, "Longitudinal dark photon", alpha=0.3, color='blue', rotation=-43, fontsize=12)
tab = np.loadtxt("model/lines/Heavy_dark_photon_Longitudinal_Dark_Photons.txt")
ax.fill(tab[:, 0], tab[:, 1], alpha=0.05, color='blue', zorder=-10)

ax.legend(frameon=1, loc='lower center', ncol=2, fontsize=12)
plt.subplots_adjust(left=0.11, right=0.95, bottom=0.09, top=0.97)

plt.savefig("./output/Dark_ALP_massless_a_noFPF.pdf")
plt.show()

## FPF

In [ ]:
setups = [
    [ "14TeV_FPF_FASER2_agamma.npy", "FPF FASER2 ($E_{a\gamma}> 0.1$ TeV)", "black", "solid", 0., 3 ],
    [ "14TeV_FPF_FASER2_aee.npy", "FPF FASER2 ($E_{ae^+e^-}> 0.1$ TeV)", "red", "solid", 0., 3 ],
    [ "14TeV_FPF_FASERnu2_agamma.npy", r"FPF FASER$\nu2$ ($E_{a\gamma}> 1$ TeV)", "black", "dotted", 0., 3 ],

    [ "14TeV_FPF_FASER2_agamma_secondary.npy", "FPF FASER2 (sec., $E_{a\gamma}> 0.1$ TeV)", "black", "dashed", 0., 3 ],
    [ "14TeV_FPF_FASERnu2_agamma_secondary.npy", r"FPF FASER$\nu2$ (sec., $E_{a\gamma}> 1$ TeV)", "black", "dashdot", 0., 3 ],
    [ "14TeV_FPF_FASERnu2_scat.npy", r"FPF FASER$\nu2$ ($e^-$ scat., dec. out.)", "gold", "solid", 0., 20 ],
    [ "14TeV_FPF_FLARE_scat.npy", r"FPF FLArE ($e^-$ scat.)", "gold", "dotted", 0., 20 ],

    [ "0.4TeV_SHiP_agamma.npy", r"SHiP ($E_{a \gamma}> 2$ GeV)", "darkgreen", "dashed", 0., 100 * 2 ],
]

In [ ]:
bounds = [
    ["Heavy_dark_photon_BaBar.txt", "BaBar", 0.55, 2.45 * 10**-3, 0],
    ["0.069TeV_NuCal_agamma.npy.txt", "NuCal", 0.012, 4.8e-1 * 10**-4, -20],
    ["0.4TeV_CHARM_agamma.npy.txt", "CHARM", 0.012, 7.6e-1 * 10**-4, -20],
]

In [ ]:
projections = [
    [ "Heavy_dark_photon_Belle_2_50fb_inverse.txt", "blue", "Belle2", 0.55, 9e-5, 0 ],
]

In [ ]:
plot, ax, ax2 = foresee.plot_reach(
    setups=setups,
    bounds=bounds,
    projections=projections,
    xlims=[0.001, 1],
    ylims=[5*10**-7, 10**-2],
    xlabel=r"$m_{\gamma^\prime}$ [GeV]",
    ylabel=r"$g_{a\gamma\gamma^\prime}$ [1/GeV]",
    legendloc=(1.00, 0.28),
    branchings=branchings,
    figsize=(8, 8),
)

ax.text(0.020, 2.0 * 10**-6, "SN1987", alpha=0.3, color='blue', fontsize=12)
low = np.loadtxt("model/lines/Heavy_dark_photon_Supernova_Emission_low.txt")
high = np.loadtxt("model/lines/Heavy_dark_photon_Supernova_Emission_high.txt")
x = np.geomspace(0.001, 1, num_of_couplings)
y = np.interp(x, high[:, 0], high[:, 1])
z = np.interp(x, low[:, 0], low[:, 1])
ax.fill_between(x, y, z, where=y >= z, alpha=0.05, color='blue', interpolate=True, lw=1, zorder=-10)

ax.text(0.00105, 3.0 * 10**-6, "Longitudinal dark photon", alpha=0.3, color='blue', rotation=-43, fontsize=12)
tab = np.loadtxt("model/lines/Heavy_dark_photon_Longitudinal_Dark_Photons.txt")
ax.fill(tab[:, 0], tab[:, 1], alpha=0.05, color='blue', zorder=-10)

ax.legend(frameon=1, loc='lower center', ncol=2, fontsize=12)
plt.subplots_adjust(left=0.11, right=0.95, bottom=0.09, top=0.97)

plot.savefig("./output/Dark_ALP_massless_a_FPF.pdf")
plot.show()